In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
ix = pd.IndexSlice
import numpy as np
import geopandas as gpd

from calenviroscreen_utils import *
from utils import *
import prep_data
import shared_utils

import shapely
from shapely.geometry import LineString

import os
from calitp.storage import get_fs
fs = get_fs()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
ces_df = prep_data.generate_calenviroscreen_lehd_data(prep_data.datasets)

In [3]:
ces_df['tract_type'] = ces_df['pop_sq_mi'].apply(lambda x: 'urban' if x > 2400 else 'suburban' if x > 800 else 'rural')

In [4]:
service_funding_joined = pd.read_parquet(f"{GCS_FILE_PATH}shape_frequency_funding.parquet")

In [5]:
service_funding_joined.head(3)

,calitp_itp_id,ntd_id,transit_provider,_5307_funds,_5311_funds,operating_expenses_total_2019,shape_id,day_name,departure_hour,route_id,trips_per_hour,mean_runtime_min
0,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,0,66,0,NaN
1,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,1,66,0,NaN
2,257,None,PresidioGo Shuttle,0.0,NaN,0.0,13737,Thursday,2,66,0,NaN


In [6]:
dates = get_recent_dates()
min_date = min(dates.values())
max_date = max(dates.values())

In [7]:
def line_from_shape(df):
    '''
    Convert a sequence of shape points for a single gtfs shape into a linestring geometry.
    '''
    try:
        assert df.size > 1, f'no geometry for shape {df.shape_id.iloc[0]}'
        df.shape_pt_sequence = df.shape_pt_sequence.astype('int64')
        df.sort_values(by='shape_pt_sequence', inplace=True) ##arrange, then convert to line to preserve order...
        route_line = LineString(list(df['geometry']))
        df['route_line'] = route_line
        return df
    except AssertionError as err:
        print(err)
        return df

In [8]:
def get_process_shapes():
    all_shapes = gpd.GeoDataFrame()
    for operator in service_funding_joined.calitp_itp_id.unique():

        print(operator)
        try:

            shapes = (tbl.gtfs_schedule.shapes()
                      >> select(_.calitp_itp_id, _.shape_id, _.shape_pt_lat, _.shape_pt_lon,
                               _.shape_pt_sequence)
                      >> filter(_.calitp_itp_id == int(operator))
                      >> collect()
                     )
            shapes_geo = gpd.GeoDataFrame(shapes, 
                                  geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat),
                                  crs = 'EPSG:4326').to_crs('EPSG:3310') ## https://epsg.io/3310 (meters)
            shape_lined = shapes_geo.groupby(['calitp_itp_id', 'shape_id']).apply(line_from_shape).reset_index(drop=True)
            shape_lined = shape_lined.drop_duplicates(subset=['shape_id'])
            shape_lined = shape_lined[['calitp_itp_id', 'shape_id', 'route_line']]
            shape_lined = gpd.GeoDataFrame(shape_lined, geometry=shape_lined['route_line'], crs='EPSG:3310')
            shape_lined = shape_lined.drop(columns=['route_line'])
            all_shapes = all_shapes.append(shape_lined)
        except:
            print(f'failed for operator {operator}')
    return all_shapes

In [9]:
# all_shapes = get_process_shapes()

In [10]:
# shared_utils.utils.geoparquet_gcs_export(all_shapes, GCS_FILE_PATH, 'shapes_initial')

In [11]:
# all_shapes.to_parquet('./working_shapes.parquet')
all_shapes = gpd.read_parquet(f'{GCS_FILE_PATH}shapes_initial.parquet')

In [12]:
all_shapes.head(3)

,calitp_itp_id,shape_id,geometry
0,257,13737,"LINESTRING (-216058.966 -21114.229, -216058.96..."
344,257,13738,"LINESTRING (-216058.966 -21114.229, -216058.96..."
0,259,p_110998,"LINESTRING (-194070.739 270848.067, -194110.74..."


### Categorize and intersect

In [13]:
## quick fix for invalid geometries?
ces_df.geometry = ces_df.geometry.buffer(0)

In [14]:
category_dissolved = ces_df.dissolve(by='tract_type')

In [15]:
def generate_shape_categories(shapes_df):
    shapes_df = shapes_df.reset_index(drop=True)

    urban = shapes_df.clip(category_dissolved.loc[['urban']])
    suburban = shapes_df.clip(category_dissolved.loc[['suburban']])
    rural = shapes_df.clip(category_dissolved.loc[['rural']])

    shapes_df['pct_urban'] = urban.geometry.length / shapes_df.geometry.length
    shapes_df['pct_suburban'] = suburban.geometry.length / shapes_df.geometry.length
    shapes_df['pct_rural'] = rural.geometry.length / shapes_df.geometry.length

    shapes_df['pct_max'] = shapes_df[['pct_urban', 'pct_suburban', 'pct_rural']].max(axis=1)
    
    return shapes_df

In [16]:
def categorize_shape(row):
    if row.pct_urban == row.pct_max:
        row['tract_type'] = 'urban'
    elif row.pct_suburban == row.pct_max:
        row['tract_type'] = 'suburban'
    elif row.pct_rural == row.pct_max:
        row['tract_type'] = 'rural'
    else:
        row['tract_type'] = np.nan
    return row

In [17]:
# processed_shapes = generate_shape_categories(all_shapes)

# processed_shapes = processed_shapes.apply(categorize_shape, axis=1)

# shared_utils.utils.geoparquet_gcs_export(processed_shapes, GCS_FILE_PATH, 'shapes_processed')

In [18]:
processed_shapes = gpd.read_parquet(f'{GCS_FILE_PATH}shapes_processed.parquet')

In [19]:
processed_shapes.head(3)

,calitp_itp_id,shape_id,geometry,pct_urban,pct_suburban,pct_rural,pct_max,tract_type
0,257,13737,"LINESTRING (-216058.966 -21114.229, -216058.96...",0.799793,0.165513,NaN,0.799793,urban
1,257,13738,"LINESTRING (-216058.966 -21114.229, -216058.96...",NaN,1.000000,NaN,1.000000,suburban
2,259,p_110998,"LINESTRING (-194070.739 270848.067, -194110.74...",NaN,0.591538,0.40818,0.591538,suburban
